# DC MOTOR 2 CLICK 

## Libraries and FPGA image

We will start by importing the *rp* (Red Pitaya) and *rp_overlay* libraries, loading the *v0.94* FPGA image, and initializing the Red Pitaya.

In [ ]:
import time
from rp_overlay import overlay
import rp

# Initialize the FPGA overlay
fpga = overlay()
rp.rp_Init()

## Macros
Here is a complete list of macros that will come in handy when customising this notebook and operating the click shields.

- **States** - RP_LOW, RP_HIGH
- **Directions** - RP_IN, RP_OUT
- **LEDs** - RP_LED0, RP_LED1, ..., RP_LED7
- **DIOx_P** - RP_DIO0_P, RP_DIO1_P, ..., RP_DIO7_P &emsp;&emsp; *# Goes up to 9 on SDRlab and STEMlab 4-Input*
- **DIOx_N** - RP_DIO0_N, RP_DIO1_N, ..., RP_DIO7_N &emsp;&thinsp; *# Goes up to 9 on SDRlab and STEMlab 4-Input*
- **Analog outputs** - RP_AOUT0, RP_AOUT1, ..., RP_AOUT3
- **Analog inputs** - RP_AIN0, RP_AIN1, ..., RP_AIN3

### Click shield macros
For easier work with the click shields, here are some macro rewrites. Marcos ending in "1" are on MicroBus1, and macros ending in "2" are on MicroBus2.

- **AN1, AIN2** - RP_AIN0, RP_AIN1
- **RST1, RST2** - RP_DIO2_N, RP_DIO4_N
- **PWM1, PWM2** - RP_DIO1_P, RP_DIO3_P
- **INT1, INT2** - RP_DIO2_P, RP_DIO4_P
- **UART_SW** - RP_DIO5_N
- **SPI_CS1, SPI_CS2** - RP_DIO1_N, RP_DIO3_N
- **Logic analyser** - RP_DIO0_P, RP_DIO1_P, ..., RP_DIO7_P

Setting up click shield macros:

In [ ]:
AN1 = rp.RP_AIN0
AN2 = rp.RP_AIN1
RST1 = rp.RP_DIO2_N
RST2 = rp.RP_DIO4_N
PWM1 = rp.RP_DIO1_P
PWM2 = rp.RP_DIO3_P
INT1 = rp.RP_DIO2_P
INT2 = rp.RP_DIO4_P
UART_SW = rp.RP_DIO5_N
SPI_CS1 = rp.RP_DIO1_N
SPI_CS2 = rp.RP_DIO3_N

# Set the direction of mikrobus pins
INPUT1 = SPI_CS1
INPUT2 = RST1
SLEEP = INT1

rp.rp_DpinSetDirection(PWM1, rp.RP_OUT)
rp.rp_DpinSetDirection(INPUT1, rp.RP_OUT)
rp.rp_DpinSetDirection(INPUT2, rp.RP_OUT)
rp.rp_DpinSetDirection(SLEEP, rp.RP_OUT)

In [ ]:
def pwm(pin, duty_cycle, num_seconds):
    period_us = 875
    pulse_us = int((duty_cycle * period_us) / 100)
    num_periods = 0
    
    while True:
        if num_periods ==  num_seconds *1000:
            break
        rp.rp_DpinSetState(pin, rp.RP_HIGH)
        time.sleep(pulse_us / 1000000)
        rp.rp_DpinSetState(pin, rp.RP_LOW)
        time.sleep((period_us - pulse_us) / 1000000)
        num_periods += 1

def set_motor_mode(motor_mode):
    if motor_mode == "MODE_CCW":
        rp.rp_DpinSetState(INPUT1, rp.RP_LOW)
        rp.rp_DpinSetState(INPUT2, rp.RP_HIGH)
        rp.rp_DpinSetState(SLEEP, rp.RP_HIGH)
    elif motor_mode == "MODE_CW":
        rp.rp_DpinSetState(INPUT1, rp.RP_HIGH)
        rp.rp_DpinSetState(INPUT2, rp.RP_LOW)
        rp.rp_DpinSetState(SLEEP, rp.RP_HIGH)
    elif motor_mode == "MODE_STOP":
        rp.rp_DpinSetState(INPUT1, rp.RP_LOW)
        rp.rp_DpinSetState(INPUT2, rp.RP_LOW)
        rp.rp_DpinSetState(SLEEP, rp.RP_HIGH)
    elif motor_mode == "MODE_STANDBY":
        rp.rp_DpinSetState(INPUT1, rp.RP_LOW)
        rp.rp_DpinSetState(INPUT2, rp.RP_LOW)
        rp.rp_DpinSetState(SLEEP, rp.RP_LOW)
    else:
        # Handle the default case
        pass

def pwm_sweep(pin, sweep_time, up_or_down):
    period_us = 875
    num_steps = int(sweep_time * 1000)  # Convert sweep time to milliseconds
    
    if up_or_down == "up":
        # The 0.15 is because the motor starts spinning at 15%
        for i in range(int(0.15 * num_steps), num_steps + 1):
            duty_cycle = (i * 100) / num_steps  # percent of power, zero to 100
            pulse_us = (duty_cycle * period_us) / 100
        
            # set pin state high
            rp.rp_DpinSetState(pin, rp.RP_HIGH) 
            # delay for pulse duration
            time.sleep(pulse_us / 1000000)

            # set pin state low
            rp.rp_DpinSetState(pin, rp.RP_LOW)

            # delay for remaining period
            time.sleep((period_us - pulse_us) / 1000000)
    
    elif up_or_down == "down":
        # The 0.15 is because the motor starts spinning at 15%
        for i in range(num_steps, int(0.15 * num_steps) - 1, -1):
            duty_cycle = (i * 100) / num_steps  # percent of power, zero to 100
            pulse_us = (duty_cycle * period_us) / 100
        
            # set pin state high
            rp.rp_DpinSetState(pin, rp.RP_HIGH)
            
            # delay for pulse duration
            time.sleep(pulse_us / 1000000)

            # set pin state low
            rp.rp_DpinSetState(pin, rp.RP_LOW)

            # delay for remaining period
            time.sleep((period_us - pulse_us) / 1000000)


The example performs a PWM sweep to gradually increase and decrease the motor speed over a 10-second period. Afterwards, it alternates the motor's direction and runs at 50% duty cycle for 3 seconds each in counter-clockwise, stopped, clockwise, and standby modes. The motor direction is set using an H-Bridge driver.

In [ ]:
#sweep up or down to lower the current spike
set_motor_mode("MODE_CW")
pwm_sweep(PWM1, 10, "up")

while True:
    # Set motor mode to counter-clockwise
    set_motor_mode("MODE_CCW")
    # Run motor at 50% duty cycle for 3 seconds
    pwm(PWM1, 50, 3)

    # Set motor mode to stop
    set_motor_mode("MODE_STOP")
    # Run motor at 50% duty cycle for 3 seconds
    pwm(PWM1, 50, 3)

    # Set motor mode to clockwise
    set_motor_mode("MODE_CW")
    # Run motor at 50% duty cycle for 3 seconds
    pwm(PWM1, 50, 3)

    # Set motor mode to standby
    set_motor_mode("MODE_STANDBY")
    # Run motor at 50% duty cycle for 3 seconds
    pwm(PWM1, 50, 3)

# Release resources
rp.rp_Release()